In [1]:
import socket
import time
import json
import threading
import pandas as pd

# This Notebook contains our Recursive Resolver

The recursive resolver should listen to 127.0.0.10 and UDP Port 53053

#### Packet Format

DNS server and client should exchange UDP packets. The transport layer payload should
be JSON strings, to ease implementation (e.g., to be independent of byte ordering). The
following keys be at least supported:
`
    dns.flags.response
    dns.flags.recdesired
    dns.qry.name
    dns.qry.type
    dns.flags.rcode
    dns.count.answers
    dns.flags.authoritative
    dns.a
    dns.ns
    dns.resp.ttl
    dns.srv.name
    dns.srv.port
    dns.srv.proto
    dns.srv.service
    dns.srv.target
`<br>
For further details see https://www.wireshark.org/docs/dfref/d/dns.html.

In [2]:
payload_dict ={  
    "dns.flags.response": 1,
    "dns.flags.recdesired": "placeholder",
    "dns.qry.name": "placeholder",
    "dns.qry.type": "placeholder",
    "dns.flags.rcode": "placeholder",
    "dns.count.answers": "placeholder",
    "dns.flags.authoritative": "placeholder",
    "dns.a": "127.0.0.10",
    "dns.ns": "www.switch.telematik",
    "dns.resp.ttl": 1234,
    "dns.srv.name": "placeholder",
    "dns.srv.port": "placeholder",
    "dns.srv.proto": "placeholder",
    "dns.srv.service": "placeholder",
    "dns.srv.target": "placeholder"
}  

In [3]:
#convert dict to json object
payload_json = json.dumps(payload_dict, indent = 4)

#### Start a tread and wait for packages for specified number of seconds

In [4]:
log_entry = '|'.join(str(x) for x in json.loads(payload_json))

In [5]:
class Cache(object):
    cache = None
    
    def __init__(self, seconds=2000, addr='127.0.0.10', port=53053):
        telematik_dns_dict = {'ns': [], 'a': [], 'ttl':[], 'authoritative': []}
        self.cache = pd.DataFrame(telematik_dns_dict, columns = ['ns','a','ttl', 'authoritative' ])
    
    #returns a requested enry from cache. Returns None if entry is not in cache.
    def get_entry(self, ns):
        entry = None
        
        if(any( self.cache.ns == ns )):
            entry = self.cache[self.cache.ns == ns]
            entry = entry.to_json()

        return entry
    
    def add_entry_to_cache(self, ns, a, authoritative):
        #ttl= 5sec. = 0.001* 5000ms
        entry = {'ns':ns, 'a':a, 'authoritative': authoritative, 'ttl':5000}
        self.cache = self.cache.append(entry, ignore_index=True)
        print('added new entry to cache: ',self.cache)
        
    def remove_entry_from_cache(self, ns):
        self.cache.drop(self.cache[self.cache['ns'] == ns].index, inplace = True) 
        print('dropped entry from cache: ',self.cache)
        
    def decrement_cache_ttls(self):
        #if ther is at least 1 cache entry decrement the ttl of all 
        if(len(self.cache) > 0 ):
            self.cache['ttl'] -= 1
            
    def check_cache_ttl_expiry(self):
        #if ther is at least 1 cache entry check all cache entry for ttl expiry
        if(len(self.cache) > 0 ):
            for i, ttl in enumerate(self.cache['ttl']):
                if(ttl<= 0):
                    self.cache.drop(i, inplace = True) 
    
    def start(self):

        start = time.time()
        time.perf_counter()    
        elapsed = 0
 
        while True:
            time.sleep(0.001)
            
            #decrement ttls -1ms
            self.decrement_cache_ttls()
            
            #check ttls and remove entry if ttl expired
            self.check_cache_ttl_expiry()
            

In [6]:
class Recursive_resolver(object):
    cache = None
    socket = None
    address = None
    port = None
    stop_time = 0
    current_stub_addr = None

    def __init__(self, seconds=2000, addr='127.0.0.10', port=53053):
        self.stop_time = seconds
        self.cache = Cache(seconds=600)   
        self.address = addr
        self.port = port
        self.dns_ip = '127.0.0.12'
        self.dns_port = 53053
        self.socket = socket.socket(socket.AF_INET,
                     socket.SOCK_DGRAM)
    
    #adds a log entry to the log file
    def add_log_entry(payload):
        #change to desired format
        log_entry = '|'.join(str(x) for x in payload_json)

        print('new log entry ',log_entry)
        
    def store_stub_request(self, stub_addr):
        self.current_stub_addr = stub_addr
     
    def send_request(self, ip, port, payload):
        if(ip == '127.0.0.9'):
            print('sendet and client ',ip)
        ##     
        self.socket.sendto(bytes(payload, 'utf-8'),(ip,port))

    def start(self):
        
        ### start the resolvers cache
        cache_thread = threading.Thread(target=self.cache.start)
        cache_thread.start()

        ### bind socket
        self.socket.bind((self.address, self.port))

        print('Resolver start listening for ',self.stop_time,' seconds...')
        start = time.time()
        time.perf_counter()    
        elapsed = 0
 
        while(elapsed < self.stop_time):
            elapsed = time.time() - start
            time.sleep(0.5)
            
            # Listen for receiving packages 
            payload_from_dns_json, client_addr = self.socket.recvfrom(1024)
            
            client_ip = client_addr[0]
            client_port = client_addr[1]
            print("client_adrr",client_addr)
            payload_from_dns = json.loads(payload_from_dns_json)
    
            #check if it is a request from a stub resolver?
            if(payload_from_dns['dns.flags.response'] == 0):
                
                #print('HERE1: ',client_addr,'  ',payload_from_dns['dns.flags.response'])
                #print('new request from a stub resolver: ',client_addr,'\n')
                self.store_stub_request(client_addr)
                cache_entry = self.cache.get_entry(payload_from_dns['dns.qry.name'])
                
                # check if requested entry is in cache?
                if( cache_entry != None ):
                    #respond to client diectly
                    #print('I HAVE THE ANSWER NO NEED TO ASK ',self.cache.cache[self.cache.cache.ns == payload_from_dns['dns.qry.name']]['ttl'] )
                    #send response from cache to stub resolver
                    #print("cache_entry",cache_entry)
                    self.socket.sendto(bytes(cache_entry, 'utf-8'), client_addr)
                else:
                     #forward to dns
                    #print('I dont know this adress and asking the dns ')
                    self.send_request(self.dns_ip, self.dns_port, json.dumps(payload_from_dns))
                    
            elif(payload_from_dns['dns.flags.response'] == 1):
                #print('HERE: ',client_addr)
                
                #response is an authoritative answer so it can be stored in the cache and send to the stub resolver
                if(payload_from_dns['dns.flags.authoritative'] == 1):
                    ip = payload_from_dns['dns.a']
                    port = 53053
                    print('RESPONSE IS AUTHORITATIVE ANSER send response to STUB: ',ip,'  ',port,'  ',payload_from_dns)
                    if(not any( self.cache.cache.ns == payload_from_dns['dns.qry.name'] )):
                        #add new entry to cache
                        self.cache.add_entry_to_cache(payload_from_dns['dns.qry.name'], 
                                                      payload_from_dns['dns.a'],
                                                      payload_from_dns['dns.flags.response'],
                                                      payload_from_dns['dns.flags.authoritative']) 
                    
                        payload_to_client = json.dumps(payload_from_dns)
                    #print('SENDING response to client: ',self.current_stub_addr)
                    if(self.current_stub_addr != None):
                        self.socket.sendto(bytes(payload_to_client, 'utf-8'), self.current_stub_addr)
                else:
                    #resonse is not an authoritative answer so we need to as the next dns server
                    #print('REsponse addr: ',client_addr,'   payload: ',payload_from_dns)
                    ip = payload_from_dns['dns.a']
                    payload_from_dns[]
                    port = 53053
                    print('RESPONSE IS NOT AUTHORITATIVE ANSER contact next server: ',ip,'  ',port,'  ',payload_from_dns)
                    payload_from_dns = json.dumps(payload_from_dns)
                    self.send_request( ip, port, payload_from_dns)
            else:
                print('response code is not 0 or 1')
                    
            print('Received package from: ',client_addr,' with payload:\n',json.dumps(payload_from_dns, indent = 4))
     
        print('Resolver stopped listening after ', elapsed ,' seconds.')

#### Create a recursive resolver object

In [7]:
resolver = Recursive_resolver(seconds=600)

#### Start the resolver as a thread and start listen

#### Send request to authorative dns server

In [8]:
stub_resolver_thread = threading.Thread(target=resolver.start)
stub_resolver_thread.start()

Resolver start listening for  600  seconds...
client_adrr ('127.0.0.9', 53053)
Received package from:  ('127.0.0.9', 53053)  with payload:
 {
    "dns.flags.response": 0,
    "dns.flags.recdesired": 1,
    "dns.qry.name": "mail.switch.telematik",
    "dns.qry.type": 1
}
client_adrr ('127.0.0.12', 53053)
RESPONSE IS NOT AUTHORITATIVE ANSER contact next server:  127.0.0.14    53053    {'dns.flags.response': 1, 'dns.flags.recavail': 0, 'dns.qry.name': 'mail.switch.telematik', 'dns.qry.type': 1, 'dns.flags.rcode': 0, 'dns.count.answers': 1, 'dns.flags.authoritative': 0, 'dns.a': '127.0.0.14', 'dns.rsp.ttl.a': 400, 'dns.ns': 'switch.telematik'}
Received package from:  ('127.0.0.12', 53053)  with payload:
 "{\"dns.flags.response\": 1, \"dns.flags.recavail\": 0, \"dns.qry.name\": \"mail.switch.telematik\", \"dns.qry.type\": 1, \"dns.flags.rcode\": 0, \"dns.count.answers\": 1, \"dns.flags.authoritative\": 0, \"dns.a\": \"127.0.0.14\", \"dns.rsp.ttl.a\": 400, \"dns.ns\": \"switch.telematik\"}"


client_adrr ('127.0.0.14', 53053)
Received package from:  ('127.0.0.14', 53053)  with payload:
 {
    "dns.flags.response": 0,
    "dns.flags.recavail": 0,
    "dns.qry.name": "mail.switch.telematik",
    "dns.qry.type": 1,
    "dns.flags.rcode": 3,
    "dns.count.answers": 1,
    "dns.flags.authoritative": 1,
    "dns.a": null,
    "dns.rsp.ttl.a": null,
    "dns.ns": null
}
client_adrr ('127.0.0.12', 53053)
RESPONSE IS NOT AUTHORITATIVE ANSER contact next server:  127.0.0.14    53053    {'dns.flags.response': 1, 'dns.flags.recavail': 0, 'dns.qry.name': 'mail.switch.telematik', 'dns.qry.type': 1, 'dns.flags.rcode': 0, 'dns.count.answers': 1, 'dns.flags.authoritative': 0, 'dns.a': '127.0.0.14', 'dns.rsp.ttl.a': 400, 'dns.ns': 'switch.telematik'}
Received package from:  ('127.0.0.12', 53053)  with payload:
 "{\"dns.flags.response\": 1, \"dns.flags.recavail\": 0, \"dns.qry.name\": \"mail.switch.telematik\", \"dns.qry.type\": 1, \"dns.flags.rcode\": 0, \"dns.count.answers\": 1, \"dns.flag

resolver.send_request('127.0.0.15',53053, payload_json)

    "dns.flags.response": 0,
    "dns.flags.recdesired": "placeholder",
    "dns.qry.name": "placeholder",
    "dns.qry.type": "placeholder",
    "dns.flags.rcode": "placeholder",
    "dns.count.answers": "placeholder",
    "dns.flags.authoritative": "placeholder",
    "dns.a": "127.0.0.10",
    "dns.ns": "www.switch.telematik",
    "dns.resp.ttl": 1234,
    "dns.srv.name": "placeholder",
    "dns.srv.port": "placeholder",
    "dns.srv.proto": "placeholder",
    "dns.srv.service": "placeholder",
    "dns.srv.target": "placeholder"

In [ ]:
{
    'dns.flags.response': 1, 
    'dns.flags.recavail': 0, 
    'dns.qry.name': 'mail.switch.telematik', 
    'dns.qry.type': 1, 
    'dns.flags.rcode': 0, 
    'dns.count.answers': 1, 
    'dns.flags.authoritative': 0, 
    'dns.a': '127.0.0.14', 
    'dns.rsp.ttl.a': 400, 
    'dns.ns': 'switch.telematik'
}

In [ ]:
    "dns.flags.response":0,
    "dns.flags.recdesired":1,
    "dns.qry.name":"mail.switch.telematik",
    "dns.qry.type":1